# O&ML Project - Notebook
This notebook is used to run the code for the O&ML project.

**Name**: Mattia Fedeli


**Student ID**: 0001148008


**Date**: 2025-06-16


**Study Course**: Automation Engineering

# Task 0: Importing necessary packages

Packages versions used in the project:

tensorflow==2.15.0

numpy==1.26.4 

pandas==2.2.2 

scikit-learn==1.4.2 

matplotlib==3.8.4

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # '0'=all, '1'=info, '2'=warnings, '3'=errors

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split


# Task 1: Data Exploration and Preprocessing

Importing the dataset from the .csv file

In [7]:
df = pd.read_csv('dataset.csv')
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0
